<a href="https://colab.research.google.com/github/jposyluzny/ENEL645/blob/main/Lectures/Week%202/DefiningFirstNeuralNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fully Connected Neural Networks

This tutorial demonstrates the usage of fully connected nueral-networks to classify 2-dimensional synthetic data. We implement the network using Keras and TensorFlow. Keras and TensorFlow are powerful frameworks that allow you to build many deep-learning-based applications. <br>
The goals of this tutorial are:


*   Introduce Keras and TensorFlow to implement fully connected neural networks
*   Illustrate the non-linear decision rules that multi-layer neural networks can achieve.

# Importing Python Libraries

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow.keras.utils import to_categorical #Function to convert labels to onehot encoding
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Input
from tensorflow.keras.optimizers import SGD

import sys

# Adding dataset_utils to the system path so we can import it
MY_UTILS_PATH = '../Modules/'

if not MY_UTILS_PATH in sys.path:
  sys.path.append(MY_UTILS_PATH)

import dataset_utils

ModuleNotFoundError: ignored